# Goals

1. prototype some pangeoLib functions
2. organize notebooks for inputs and output classification for ET

## prototype
1. boto 3 list all tiles
2. boto 3 list all outputs in tiles by subset
3. build coverage geometries for each tile day 001

In [1]:
import boto3

In [2]:
from pangeoLib.aws_authenticate import aws_authenticate

aws_authenticate()

In [3]:
# THIS code does not yet work with requestor pays - perhaps fsspec is much easier
def list_all_tiles():
    
    bucket = 'dev-et-data'
    #Make sure you provide / in the end
    prefix = 'tiles/'  

    client = boto3.client('s3')
    result = client.list_objects(Bucket=bucket, Prefix=prefix, Delimiter='/')
    for o in result.get('CommonPrefixes'):
        print ('sub folder : ', o.get('Prefix'))

In [4]:
#list_all_tiles()

In [5]:
! aws s3 ls --request-payer requester dev-et-data/tiles/

                           PRE t40N-80E/
                           PRE tile40N-100E_chip32N-100E/
                           PRE tile40N-100E_chip32N-92E/
                           PRE tile40N-100E_chip32N-94E/
                           PRE tile40N-100E_chip32N-96E/
                           PRE tile40N-100E_chip32N-98E/
                           PRE tile40N-100E_chip34N-100E/
                           PRE tile40N-100E_chip34N-92E/
                           PRE tile40N-100E_chip34N-94E/
                           PRE tile40N-100E_chip34N-96E/
                           PRE tile40N-100E_chip34N-98E/
                           PRE tile40N-100E_chip36N-100E/
                           PRE tile40N-100E_chip36N-92E/
                           PRE tile40N-100E_chip36N-94E/
                           PRE tile40N-100E_chip36N-96E/
                           PRE tile40N-100E_chip36N-98E/
                           PRE tile40N-100E_chip38N-100E/
                           PRE tile40N-100E

In [6]:
aws_authenticate()

In [7]:
! rio info /vsis3/dev-et-data/test/compressed/NDVI_filled/2001/2001001.250_m_NDVI.tif

{"blockxsize": 512, "blockysize": 512, "bounds": [-155.57238265799998, 20.000239124000082, -52.215132156499976, 49.999999996000085], "colorinterp": ["gray"], "compress": "deflate", "count": 1, "crs": "EPSG:4326", "descriptions": [null], "driver": "GTiff", "dtype": "float32", "height": 14416, "indexes": [1], "interleave": "band", "lnglat": [-103.89375740724998, 35.00011956000009], "mask_flags": [["nodata"]], "nodata": -3.4028234663852886e+38, "res": [0.0020810045, 0.0020810045], "shape": [14416, 49667], "tiled": true, "transform": [0.0020810045, 0.0, -155.57238265799998, 0.0, -0.0020810045, 49.999999996000085, 0.0, 0.0, 1.0], "units": [null], "width": 49667}


In [8]:
import fsspec
fs = fsspec.filesystem('s3', anon=False, requester_pays=True)
chip_list = fs.ls('dev-et-data/tiles/')

In [9]:
! rio info /vsis3/dev-et-data/tiles/tile40N-100E_chip32N-100E/dd_2014.tif

{"bounds": [-100.00123780499997, 29.999468047915045, -97.99931147599997, 32.001394376915044], "colorinterp": ["gray"], "count": 1, "crs": "EPSG:4326", "descriptions": [null], "driver": "GTiff", "dtype": "float64", "height": 962, "indexes": [1], "interleave": "band", "lnglat": [-99.00027464049998, 31.000431212415045], "mask_flags": [["all_valid"]], "nodata": null, "res": [0.0020810045, 0.0020810045], "shape": [962, 962], "tiled": false, "transform": [0.0020810045, 0.0, -100.00123780499997, 0.0, -0.0020810045, 32.001394376915044, 0.0, 0.0, 1.0], "units": [null], "width": 962}


In [10]:
import rasterio
file='/vsis3/dev-et-data/tiles/tile40N-100E_chip32N-100E/dd_2014.tif'
src = rasterio.open(file)
src.shape

(962, 962)

In [11]:
#dir(src)

src.bounds.left

-100.00123780499997

In [12]:
dir(src.bounds)

['__add__',
 '__class__',
 '__contains__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getnewargs__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__module__',
 '__mul__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rmul__',
 '__setattr__',
 '__sizeof__',
 '__slots__',
 '__str__',
 '__subclasshook__',
 '_asdict',
 '_field_defaults',
 '_fields',
 '_fields_defaults',
 '_make',
 '_replace',
 'bottom',
 'count',
 'index',
 'left',
 'right',
 'top']



You are almost there with what you tried, and using the box method is indeed the best way. With the list comprehension, you can do

b = [box(l, b, r, t) for l, b, r, t in zip(df.left, df.bottom, df.right, df.top)]

Another option is to apply the box function to each row of your dataframe:

b = df.apply(lambda row: box(row.left, row.bottom, row.right, row.top), axis=1)

Once you have converted the bounding boxes to polygons, make sure to actually create a GeoDataFrame:

gdf = geopandas.GeoDataFrame(df, geometry=b)



In [13]:
chip_list

['dev-et-data/tiles/t40N-80E',
 'dev-et-data/tiles/tile40N-100E_chip32N-100E',
 'dev-et-data/tiles/tile40N-100E_chip32N-92E',
 'dev-et-data/tiles/tile40N-100E_chip32N-94E',
 'dev-et-data/tiles/tile40N-100E_chip32N-96E',
 'dev-et-data/tiles/tile40N-100E_chip32N-98E',
 'dev-et-data/tiles/tile40N-100E_chip34N-100E',
 'dev-et-data/tiles/tile40N-100E_chip34N-92E',
 'dev-et-data/tiles/tile40N-100E_chip34N-94E',
 'dev-et-data/tiles/tile40N-100E_chip34N-96E',
 'dev-et-data/tiles/tile40N-100E_chip34N-98E',
 'dev-et-data/tiles/tile40N-100E_chip36N-100E',
 'dev-et-data/tiles/tile40N-100E_chip36N-92E',
 'dev-et-data/tiles/tile40N-100E_chip36N-94E',
 'dev-et-data/tiles/tile40N-100E_chip36N-96E',
 'dev-et-data/tiles/tile40N-100E_chip36N-98E',
 'dev-et-data/tiles/tile40N-100E_chip38N-100E',
 'dev-et-data/tiles/tile40N-100E_chip38N-92E',
 'dev-et-data/tiles/tile40N-100E_chip38N-94E',
 'dev-et-data/tiles/tile40N-100E_chip38N-96E',
 'dev-et-data/tiles/tile40N-100E_chip38N-98E',
 'dev-et-data/tiles/tile4

In [14]:
def get_bounding(file):
    try:
        src = rasterio.open(file)
        return(src.bounds)
    except:
        pass
        

In [15]:

import pandas as pd
df = pd.DataFrame(columns=('Chip', 'left', 'bottom', 'right', 'top'))
i=0
for chip in chip_list:
    file = '/vsis3/' + chip + '/dd_2013.tif'
    bounds = get_bounding(file)
    if not bounds is None:
        print(bounds.left)
        df.loc[i] =[chip, bounds.left, bounds.bottom, bounds.right, bounds.top]
        i=i+1

-100.00123780499997
-92.00185650699997
-94.00170183149997
-96.00154715599997
-98.00139248049997
-100.00123780499997
-92.00185650699997
-94.00170183149997
-96.00154715599997
-98.00139248049997
-100.00123780499997
-92.00185650699997
-94.00170183149997
-96.00154715599997
-98.00139248049997
-100.00123780499997
-92.00185650699997
-94.00170183149997
-96.00154715599997
-98.00139248049997
-100.00123780499997
-92.00185650699997
-94.00170183149997
-96.00154715599997
-98.00139248049997
-102.00108312949997
-102.00108312949997
-104.00092845399998
-104.00092845399998
-106.00077377849998
-106.00077377849998
-108.00061910299996
-108.00061910299996
-110.00046442749996
-110.00046442749996
-102.00108312949997
-102.00108312949997
-104.00092845399998
-104.00092845399998
-106.00077377849998
-106.00077377849998
-108.00061910299996
-108.00061910299996
-110.00046442749996
-110.00046442749996
-102.00108312949997
-102.00108312949997
-104.00092845399998
-104.00092845399998
-106.00077377849998
-106.00077377849998


In [16]:
df.head(30)

,Chip,left,bottom,right,top
0,dev-et-data/tiles/tile40N-100E_chip32N-100E,-100.001238,29.999468,-97.999311,32.001394
1,dev-et-data/tiles/tile40N-100E_chip32N-92E,-92.001857,29.999468,-89.999930,32.001394
2,dev-et-data/tiles/tile40N-100E_chip32N-94E,-94.001702,29.999468,-91.999776,32.001394
3,dev-et-data/tiles/tile40N-100E_chip32N-96E,-96.001547,29.999468,-93.999621,32.001394
4,dev-et-data/tiles/tile40N-100E_chip32N-98E,-98.001392,29.999468,-95.999466,32.001394
5,dev-et-data/tiles/tile40N-100E_chip34N-100E,-100.001238,31.999313,-97.999311,34.001240
6,dev-et-data/tiles/tile40N-100E_chip34N-92E,-92.001857,31.999313,-89.999930,34.001240
7,dev-et-data/tiles/tile40N-100E_chip34N-94E,-94.001702,31.999313,-91.999776,34.001240
8,dev-et-data/tiles/tile40N-100E_chip34N-96E,-96.001547,31.999313,-93.999621,34.001240
9,dev-et-data/tiles/tile40N-100E_chip34N-98E,-98.001392,31.999313,-95.999466,34.001240


In [17]:
dir(df)

['Chip',
 'T',
 '_AXIS_ALIASES',
 '_AXIS_IALIASES',
 '_AXIS_LEN',
 '_AXIS_NAMES',
 '_AXIS_NUMBERS',
 '_AXIS_ORDERS',
 '_AXIS_REVERSED',
 '__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_priority__',
 '__array_wrap__',
 '__bool__',
 '__class__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__delitem__',
 '__dict__',
 '__dir__',
 '__div__',
 '__doc__',
 '__eq__',
 '__finalize__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__iadd__',
 '__iand__',
 '__ifloordiv__',
 '__imod__',
 '__imul__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__ior__',
 '__ipow__',
 '__isub__',
 '__iter__',
 '__itruediv__',
 '__ixor__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__nonzero__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__rdiv__',
 '__reduce__',
 '_

In [18]:
df.describe()

,left,bottom,right,top
count,125.000000,125.000000,125.000000,125.000000
mean,-94.000953,33.999159,-91.998943,36.001085
std,12.048236,2.839590,12.048264,2.839590
min,-110.000464,29.999468,-107.998538,32.001394
25%,-104.000928,31.999313,-101.999002,34.001240
50%,-96.001547,33.999159,-93.999621,36.001085
75%,-84.000394,35.999004,-81.998468,38.000930
max,-72.001322,37.998849,-69.999396,40.000776


In [19]:
import geopandas
from shapely.geometry import box
#import shapely.bounding.box as box

#Once you have converted the bounding boxes to polygons, make sure to actually create a GeoDataFrame:
b = df.apply(lambda row: box(row.left, row.bottom, row.right, row.top), axis=1)
gdf = geopandas.GeoDataFrame(df, geometry=b)

In [20]:
gdf.head()

,Chip,left,bottom,right,top,geometry
0,dev-et-data/tiles/tile40N-100E_chip32N-100E,-100.001238,29.999468,-97.999311,32.001394,"POLYGON ((-97.99931 29.99947, -97.99931 32.001..."
1,dev-et-data/tiles/tile40N-100E_chip32N-92E,-92.001857,29.999468,-89.999930,32.001394,"POLYGON ((-89.99993 29.99947, -89.99993 32.001..."
2,dev-et-data/tiles/tile40N-100E_chip32N-94E,-94.001702,29.999468,-91.999776,32.001394,"POLYGON ((-91.99978 29.99947, -91.99978 32.001..."
3,dev-et-data/tiles/tile40N-100E_chip32N-96E,-96.001547,29.999468,-93.999621,32.001394,"POLYGON ((-93.99962 29.99947, -93.99962 32.001..."
4,dev-et-data/tiles/tile40N-100E_chip32N-98E,-98.001392,29.999468,-95.999466,32.001394,"POLYGON ((-95.99947 29.99947, -95.99947 32.001..."


In [21]:
import folium


In [22]:
tony_plot_tiles_json = gdf.to_json()

In [23]:

map_osm = folium.Map(location=[35, -90], zoom_start=5)

tile_boxes=folium.features.GeoJson(tony_plot_tiles_json)
map_osm.add_child(tile_boxes)
map_osm

In [36]:
my_s= gdf.loc[0]

#print(dir(my_s))


print(my_s.geometry)



POLYGON ((-97.99931147599997 29.99946804791504, -97.99931147599997 32.00139437691504, -100.001237805 32.00139437691504, -100.001237805 29.99946804791504, -97.99931147599997 29.99946804791504))


In [37]:
geopandas.GeoSeries([my_s.geometry]).to_json()

'{"type": "FeatureCollection", "features": [{"id": "0", "type": "Feature", "properties": {}, "geometry": {"type": "Polygon", "coordinates": [[[-97.99931147599997, 29.999468047915045], [-97.99931147599997, 32.001394376915044], [-100.00123780499997, 32.001394376915044], [-100.00123780499997, 29.999468047915045], [-97.99931147599997, 29.999468047915045]]]}, "bbox": [-100.00123780499997, 29.999468047915045, -97.99931147599997, 32.001394376915044]}], "bbox": [-100.00123780499997, 29.999468047915045, -97.99931147599997, 32.001394376915044]}'

In [43]:
map_osm = folium.Map(location=[35, -90], zoom_start=5)

style_function = lambda x: {'fillColor': 'red', 'color': 'green'} 

for index, row in gdf.iterrows():
    print(row.geometry)
    print(row.Chip)
    my_useful_geojson = geopandas.GeoSeries([row.geometry]).to_json()
    gjson = folium.GeoJson(my_useful_geojson, style_function=style_function, tooltip=row.Chip).add_to(map_osm)



map_osm

POLYGON ((-97.99931147599997 29.99946804791504, -97.99931147599997 32.00139437691504, -100.001237805 32.00139437691504, -100.001237805 29.99946804791504, -97.99931147599997 29.99946804791504))
dev-et-data/tiles/tile40N-100E_chip32N-100E
POLYGON ((-89.99993017799997 29.99946804791504, -89.99993017799997 32.00139437691504, -92.00185650699997 32.00139437691504, -92.00185650699997 29.99946804791504, -89.99993017799997 29.99946804791504))
dev-et-data/tiles/tile40N-100E_chip32N-92E
POLYGON ((-91.99977550249997 29.99946804791504, -91.99977550249997 32.00139437691504, -94.00170183149997 32.00139437691504, -94.00170183149997 29.99946804791504, -91.99977550249997 29.99946804791504))
dev-et-data/tiles/tile40N-100E_chip32N-94E
POLYGON ((-93.99962082699997 29.99946804791504, -93.99962082699997 32.00139437691504, -96.00154715599997 32.00139437691504, -96.00154715599997 29.99946804791504, -93.99962082699997 29.99946804791504))
dev-et-data/tiles/tile40N-100E_chip32N-96E
POLYGON ((-95.99946615149997 29

In [28]:
my_s= gdf.loc[0]

In [29]:
print(my_s)

Chip              dev-et-data/tiles/tile40N-100E_chip32N-100E
left                                                 -100.001
bottom                                                29.9995
right                                                -97.9993
top                                                   32.0014
geometry    POLYGON ((-97.99931147599997 29.99946804791504...
Name: 0, dtype: object


In [30]:


my_gdf = geopandas.GeoDataFrame(my_s, geometry = [[my_s.geometry,]])

TypeError: Input must be valid geometry objects: [<shapely.geometry.polygon.Polygon object at 0x7f74414ca1d0>]

In [ ]:
#a=my_gdf.to_json()

type(my_gdf)

In [ ]:
a_json = my_gdf.to_json()

In [ ]:
from geojson import Polygon

In [ ]:
print(my_s.geometry)

In [ ]:
Polygon(my_s.geometry)

In [ ]:
dir(my_s.geometry)

In [ ]:
my_s.geometry.bounds

In [ ]:
my_s.geometry.coords()

In [ ]:
print(my_s.geometry.boundary)

In [ ]:
dir(my_s.geometry.boundary)

In [ ]:
from geojson import LineString

LineString(my_s.geometry.boundary)

In [ ]:
print(my_s.geometry.boundary.xy)

In [ ]:
LineString((-103.9988474495 33.99915869691505, -103.9988474495 36.00108502591505, -106.0007737785 36.00108502591505, -106.0007737785 33.99915869691505, -103.9988474495 33.99915869691505))

In [ ]:
for index, row in gdf.iterrows():
    print(row)
    print(row.values)
    

In [ ]:
dir(row)